In [2]:
import importlib
from pathlib import Path

import instructor
import openai
import pandas as pd
from rich.console import Console

c = Console(width=60)

GDRIVE = Path("~/Google Drive/Shared drives/Solutions").expanduser()

# Quick Start

In [ ]:
from cuery import Prompt, Response, Task


# Define response model
class Entity(Response):
    name: str
    type: str


class NamedEntities(Response):
    entities: list[Entity]


context = pd.DataFrame(
    {
        "text": [
            "Apple is headquartered in Cupertino, California."
            "Barack Obama was the 44th President of the United States.",
            "The Eiffel Tower is located in Paris, France.",
        ]
    }
)

prompt = Prompt.from_string("Extract named entities from the following text: {{text}}")
task = Task(prompt=prompt, response=NamedEntities)
result = await task(context, model="openai/gpt-3.5-turbo", n_concurrent=10)
print(result.to_pandas(explode=False))

Gathering responses: 100%|██████████| 2/2 [00:01<00:00,  1.35it/s]

                                                text  \
0  Apple is headquartered in Cupertino, Californi...   
1      The Eiffel Tower is located in Paris, France.   

                                            entities  
0  [name='Apple' type='organization', name='Cuper...  
1  [name='Eiffel Tower' type='landmark', name='Pa...  


In [38]:
print(result.to_pandas(explode=True))

                                                text           name  \
0   Apple is headquartered in Cupertino, California.          Apple   
1   Apple is headquartered in Cupertino, California.      Cupertino   
2   Apple is headquartered in Cupertino, California.     California   
3  Barack Obama was the 44th President of the Uni...   Barack Obama   
4  Barack Obama was the 44th President of the Uni...           44th   
5  Barack Obama was the 44th President of the Uni...  United States   
6      The Eiffel Tower is located in Paris, France.   Eiffel Tower   
7      The Eiffel Tower is located in Paris, France.          Paris   
8      The Eiffel Tower is located in Paris, France.         France   

           type  
0  Organization  
1      Location  
2      Location  
3        Person  
4       Ordinal  
5      Location  
6      Location  
7      Location  
8      Location  


# Prompts

In [3]:
from cuery import Prompt

prompt = Prompt.from_string("Explain {{ topic }} in simple terms.")
c.print(prompt)

╭───────────────────────── Prompt ─────────────────────────╮
│                                                          │
│  Required: ['topic']                                     │
│                                                          │
│ ╭──────────────────────── USER ────────────────────────╮ │
│ │ Explain {{ topic }} in simple terms.                 │ │
│ ╰──────────────────────────────────────────────────────╯ │
╰──────────────────────────────────────────────────────────╯

# Context

In [ ]:
from cuery.context import iter_context

df = pd.DataFrame(
    {
        "topic": ["Machine Learning", "Natural Language Processing", "Computer Vision"],
        "audience": ["beginners", "developers", "researchers"],
    }
)

contexts, count = iter_context(df, required=["topic", "audience"])
next(contexts)

# Response, Task

In [11]:
from cuery import Field, Prompt, Response, Task


class MovieRecommendation(Response):
    title: str
    year: int = Field(gt=1900, lt=2030)
    genre: list[str]
    rating: float = Field(ge=0, le=10)


# Multi-output response model
class MovieRecommendations(Response):
    recommendations: list[MovieRecommendation]


movie_prompt = Prompt.from_string(
    "Recommend a movie for {{ audience }} interested in {{ topic }}."
)
context = [
    {"topic": "Machine Learning", "audience": "beginners"},
    {"topic": "Computer Vision", "audience": "researchers"},
]

movie_task = Task(prompt=movie_prompt, response=MovieRecommendations)
result = await movie_task(context)
print(result)
print(result.to_pandas())

Iterating context:   0%|          | 0/2 [00:00<?, ?it/s]

[MovieRecommendations(recommendations=[MovieRecommendation(title='The Matrix', year=1999, genre=['Action', 'Sci-Fi'], rating=8.7), MovieRecommendation(title='Ex Machina', year=2014, genre=['Drama', 'Sci-Fi'], rating=7.7), MovieRecommendation(title='Her', year=2013, genre=['Drama', 'Romance', 'Sci-Fi'], rating=8.0)]), MovieRecommendations(recommendations=[MovieRecommendation(title='Her', year=2013, genre=['Drama', 'Romance', 'Sci-Fi'], rating=8.0), MovieRecommendation(title='Blade Runner 2049', year=2017, genre=['Drama', 'Mystery', 'Sci-Fi'], rating=8.0), MovieRecommendation(title='Ex Machina', year=2014, genre=['Drama', 'Sci-Fi', 'Thriller'], rating=7.7)])]
              topic     audience              title  year  \
0  Machine Learning    beginners         The Matrix  1999   
1  Machine Learning    beginners         Ex Machina  2014   
2  Machine Learning    beginners                Her  2013   
3   Computer Vision  researchers                Her  2013   
4   Computer Vision  research

In [12]:
print(result.usage())

   prompt  completion      cost
0     147          79  0.000192
1     147          90  0.000208


In [23]:
from cuery import pprint

print(result[0]._raw_response.model)
print(result[0]._raw_response.service_tier)

gpt-3.5-turbo-0125
default


In [30]:
print(movie_task.query_log.queries[0]["messages"][0]["content"])

Recommend a movie for beginners interested in Machine Learning.


In [6]:
c.print(result[0])

╭───────────── RESPONSE: MovieRecommendations ─────────────╮
│                                                          │
│ ╭─ recommendations: list[__main__.MovieRecommendation]─╮ │
│ │                                                      │ │
│ │  {'required': True}                                  │ │
│ │                                                      │ │
│ ╰──────────────────────────────────────────────────────╯ │
│                                                          │
│  ╭───────── RESPONSE: MovieRecommendation ──────────╮    │
│  │                                                  │    │
│  │ ╭─ title: str ─────────────────────────────────╮ │    │
│  │ │                                              │ │    │
│  │ │  {'required': True}                          │ │    │
│  │ │                                              │ │    │
│  │ ╰──────────────────────────────────────────────╯ │    │
│  │ ╭─ year: int ──────────────────────────────────╮ │    │
│  │ │                                              │ │    │
│  │ │  {                                           │ │    │
│  │ │      'required': True,                       │ │    │
│  │ │      'metadata': [                           │ │    │
│  │ │          Gt(gt=1900),                        │ │    │
│  │ │          Lt(lt=2030)                         │ │    │
│  │ │      ]                                       │ │    │
│  │ │  }                                           │ │    │
│  │ │                                              │ │    │
│  │ ╰──────────────────────────────────────────────╯ │    │
│  │ ╭─ genre: list[str] ───────────────────────────╮ │    │
│  │ │                                              │ │    │
│  │ │  {'required': True}                          │ │    │
│  │ │                                              │ │    │
│  │ ╰──────────────────────────────────────────────╯ │    │
│  │ ╭─ rating: float ──────────────────────────────╮ │    │
│  │ │                                              │ │    │
│  │ │  {                                           │ │    │
│  │ │      'required': True,                       │ │    │
│  │ │      'metadata': [Ge(ge=0), Le(le=10)]       │ │    │
│  │ │  }                                           │ │    │
│  │ │                                              │ │    │
│  │ ╰──────────────────────────────────────────────╯ │    │
│  │                                                  │    │
│  ╰──────────────────────────────────────────────────╯    │
│                                                          │
│                                                          │
╰──────────────────────────────────────────────────────────╯

# Chain

In [ ]:
from typing import Literal

from cuery import Chain, Task

# Reuse example from above
movies = Task(prompt=movie_prompt, response=MovieRecommendations)


# Add PG rating
class Rating(Response):
    pg_category: Literal["G", "PG", "PG-13", "R", "NC-17"] = Field(
        ..., description="PG rating of the movie."
    )
    pg_reason: str = Field(..., description="Reason for the rating.")


rating_prompt = Prompt.from_string("What is the PG rating for {{ title }}?")
rating_task = Task(prompt=rating_prompt, response=Rating)

# Create a chain of tasks
chain = Chain(movie_task, rating_task)
result = await chain(context, model="openai/gpt-3.5-turbo", n_concurrent=20)
result

[06/04/25 17:03:37] INFO     [1/2] Running task 'MovieRecommendations'                                  ]8;id=315752;file:///Users/thomas/code/cuery/src/cuery/task.py\task.py]8;;\:]8;id=703029;file:///Users/thomas/code/cuery/src/cuery/task.py#205\205]8;;\

Gathering responses: 100%|██████████| 2/2 [00:02<00:00,  1.40s/it]


[06/04/25 17:03:40] INFO     [2/2] Running task 'Rating'                                                ]8;id=608666;file:///Users/thomas/code/cuery/src/cuery/task.py\task.py]8;;\:]8;id=200242;file:///Users/thomas/code/cuery/src/cuery/task.py#205\205]8;;\

Gathering responses: 100%|██████████| 6/6 [00:02<00:00,  2.84it/s]


,topic,audience,title,year,genre,rating,pg_category,pg_reason
0,Machine Learning,beginners,The Matrix,1999,"[Action, Sci-Fi]",8.7,R,Violence and language
1,Machine Learning,beginners,Ex Machina,2014,"[Drama, Sci-Fi]",7.7,R,"Strong language, graphic nudity, and sexual co..."
2,Machine Learning,beginners,Ex Machina,2014,"[Drama, Sci-Fi]",7.7,R,"Rated R for graphic nudity, language, sexual r..."
3,Machine Learning,beginners,Her,2013,"[Drama, Romance, Sci-Fi]",8.0,R,"Brief graphic nudity, Sexual content and language"
4,Machine Learning,beginners,Her,2013,"[Drama, Romance, Sci-Fi]",8.0,R,"Language, sexual content and brief graphic nudity"
5,Computer Vision,researchers,Blade Runner,1982,"[Sci-Fi, Thriller]",8.1,R,"Violence, some language, and brief nudity"
6,Computer Vision,researchers,Her,2013,"[Drama, Romance, Sci-Fi]",8.0,R,"Brief graphic nudity, Sexual content and language"
7,Computer Vision,researchers,Her,2013,"[Drama, Romance, Sci-Fi]",8.0,R,"Language, sexual content and brief graphic nudity"
8,Computer Vision,researchers,Ex Machina,2014,"[Drama, Sci-Fi, Thriller]",7.7,R,"Strong language, graphic nudity, and sexual co..."
9,Computer Vision,researchers,Ex Machina,2014,"[Drama, Sci-Fi, Thriller]",7.7,R,"Rated R for graphic nudity, language, sexual r..."


In [9]:
print(result)

              topic     audience         title  year  \
0  Machine Learning    beginners    The Matrix  1999   
1  Machine Learning    beginners    Ex Machina  2014   
2  Machine Learning    beginners    Ex Machina  2014   
3  Machine Learning    beginners           Her  2013   
4  Machine Learning    beginners           Her  2013   
5   Computer Vision  researchers  Blade Runner  1982   
6   Computer Vision  researchers           Her  2013   
7   Computer Vision  researchers           Her  2013   
8   Computer Vision  researchers    Ex Machina  2014   
9   Computer Vision  researchers    Ex Machina  2014   

                       genre  rating pg_category  \
0           [Action, Sci-Fi]     8.7           R   
1            [Drama, Sci-Fi]     7.7           R   
2            [Drama, Sci-Fi]     7.7           R   
3   [Drama, Romance, Sci-Fi]     8.0           R   
4   [Drama, Romance, Sci-Fi]     8.0           R   
5         [Sci-Fi, Thriller]     8.1           R   
6   [Drama, Romance